In [1]:
!curl -LsSf https://astral.sh/uv/install.sh | sh
!source $HOME/.cargo/env

!uv pip install --force-reinstall "numpy>=1.24,<1.27"

# Step 2: Numba 재설치 (NumPy 버전 확인 후)
!uv pip install --force-reinstall numba

# Step 3: 나머지 패키지들 설치
!uv pip install openai-whisper datasets jiwer

# Step 4: Transformers는 --no-deps로 (NumPy 업그레이드 방지)
!uv pip install --no-deps transformers tokenizers huggingface-hub safetensors


downloading uv 0.7.12 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
/bin/bash: line 1: /root/.cargo/env: No such file or directory
Using Python 3.11.13 environment at: /usr
Resolved 1 package in 50ms
Prepared 1 package in 0.35ms
Uninstalled 1 package in 86ms
Installed 1 package in 64ms
 - numpy==2.2.6
 + numpy==1.26.4
Using Python 3.11.13 environment at: /usr
Resolved 3 packages in 55ms
Prepared 3 packages in 0.50ms
Uninstalled 3 packages in 85ms
Installed 3 packages in 121ms
 ~ llvmlite==0.44.0
 ~ numba==0.61.2
 - numpy==1.26.4
 + numpy==2.2.6
Using Python 3.11.13 environment at: /usr
Audited 3 packages in 253ms
Using Python 3.11.13 environment at: /usr
Audited 4 packages in 204ms


In [2]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name()}")


CUDA available: True
CUDA device count: 1
Current CUDA device: 0
CUDA device name: Tesla T4


In [30]:
from transformers import pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# HF Whisper 파이프라인 로드
print("🔄 HF Whisper 모델 로딩 중...")
hf_pipe = pipeline(
    "automatic-speech-recognition",
    model="JayYH/whisper-medium-ko",
    device=device,
    generate_kwargs={"language": "korean", "task": "transcribe"}
)

🔄 HF Whisper 모델 로딩 중...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
import numpy as np
print(np.__version__)

import whisper

# 모델 로드 (자동으로 다운로드됨)
model = whisper.load_model("base")

print("✅ Whisper 모델 로드 완료!")
print("📱 사용 가능한 모델 크기: tiny, base, small, medium, large")


2.2.6
✅ Whisper 모델 로드 완료!
📱 사용 가능한 모델 크기: tiny, base, small, medium, large


In [19]:
from datasets import load_dataset
import os
import shutil
# 🔥 캐시 디렉토리 완전 삭제
cache_dir = "/root/.cache/huggingface/datasets/Bingsu___zeroth-korean"
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print("🗑️ 기존 캐시 삭제 완료")

# 새로 다운로드
test_dataset = load_dataset("Bingsu/zeroth-korean", split="test[:10]")
print(f"✅ Zeroth-Korean 데이터 로드 성공: {len(test_dataset)}개 샘플")


🗑️ 기존 캐시 삭제 완료


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/22263 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/457 [00:00<?, ? examples/s]

✅ Zeroth-Korean 데이터 로드 성공: 10개 샘플


In [32]:

from tqdm import tqdm
import jiwer

def test_whisper_simple(model, dataset, max_samples=50):
    """OpenAI Whisper 간단 테스트 (jiwer 사용)"""

    if dataset is None:
        print("⚠️ 데이터셋이 없어서 테스트를 건너뜁니다")
        return

    predictions = []
    references = []

    print(f"🔍 {max_samples}개 샘플로 테스트 중...")

    # 샘플 선택
    test_samples = dataset.select(range(min(max_samples, len(dataset))))

    for example in test_samples:
        try:
            # 오디오 데이터
            audio = example["audio"]["array"]

            if hasattr(audio, 'dtype'):
                audio = audio.astype('float32')


            # Whisper로 전사 (한 줄!)
            result = model.transcribe(
                audio,
                language="ko",
                fp16=False,      # CPU에서 fp16 비활성화 (타입 에러 방지)
                verbose=False    # 로그 최소화
            )
            prediction = result["text"]

            reference = example["text"]

            predictions.append(prediction)
            references.append(reference)

        except Exception as e:
            print(f"❌ 오류: {e}")
            continue

    if len(predictions) == 0:
        print("⚠️ 처리된 샘플이 없습니다")
        return

    # jiwer로 정확한 성능 계산
    cer = jiwer.cer(references, predictions) * 100
    wer = jiwer.wer(references, predictions) * 100

    print(f"\n🎯 성능 결과:")
    print(f"📈 CER: {cer:.2f}%")
    print(f"📈 WER: {wer:.2f}%")

    # 샘플 결과 출력
    print(f"\n📋 샘플 결과:")
    for i in range(len(predictions)):
        print(f"\n샘플 {i+1}:")
        print(f"🎯 정답: {references[i]}")
        print(f"🤖 예측: {predictions[i]}")

    return {"cer": cer, "wer": wer, "predictions": predictions, "references": references}

def test_hf_model(pipe, dataset, max_samples=50):
    """Hugging Face 모델 테스트 (OpenAI Whisper와 동일한 구조)"""
    if dataset is None:
        print("⚠️ 데이터셋이 없어서 테스트를 건너뜁니다")
        return

    predictions = []
    references = []

    print(f"🔍 {max_samples}개 샘플로 테스트 중...")

    # 샘플 선택
    test_samples = dataset.select(range(min(max_samples, len(dataset))))

    for example in test_samples:
        try:
            # 오디오 데이터
            audio = example["audio"]["array"]
            sample_rate = example["audio"]["sampling_rate"]

            # HF 파이프라인으로 전사 (직접 호출!)
            result = pipe(audio)  # 🔥 이렇게 직접 호출
            prediction = result["text"]

            reference = example["text"]

            predictions.append(prediction)
            references.append(reference)

        except Exception as e:
            print(f"❌ 오류: {e}")
            continue

    if len(predictions) == 0:
        print("⚠️ 처리된 샘플이 없습니다")
        return

    # jiwer로 정확한 성능 계산
    cer = jiwer.cer(references, predictions) * 100
    wer = jiwer.wer(references, predictions) * 100

    print(f"\n🎯 성능 결과:")
    print(f"📈 CER: {cer:.2f}%")
    print(f"📈 WER: {wer:.2f}%")

    # 샘플 결과 출력
    print(f"\n📋 샘플 결과:")
    for i in range(len(predictions)):
        print(f"\n샘플 {i+1}:")
        print(f"🎯 정답: {references[i]}")
        print(f"🤖 예측: {predictions[i]}")

    return {"cer": cer, "wer": wer, "predictions": predictions, "references": references}


In [33]:
all_results = {}

# OpenAI Whisper 테스트
print(f"\n🔄 OpenAI Whisper base 모델 테스트 중...")
if test_dataset:
    results = test_whisper_simple(model, test_dataset, max_samples=10)
    if results:
        all_results["OpenAI Whisper"] = results

# HF Whisper 테스트
print(f"\n🔄 HF Whisper base 모델 테스트 중...")
if test_dataset:
    results = test_hf_model(hf_pipe, test_dataset, max_samples=10)
    if results:
        all_results["HF Whisper"] = results

# 🔥 CHANGED: 간단한 비교 결과
if all_results:
    print(f"\n📊 OpenAI vs HF 모델 비교:")
    print("=" * 40)
    for model_name, result in all_results.items():
        print(f"{model_name:15} | CER: {result['cer']:5.1f}% | WER: {result['wer']:5.1f}%")


🔄 OpenAI Whisper base 모델 테스트 중...
🔍 10개 샘플로 테스트 중...


100%|██████████| 926/926 [00:00<00:00, 2344.03frames/s]
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



🎯 성능 결과:
📈 CER: 12.27%
📈 WER: 34.36%

📋 샘플 결과:

샘플 1:
🎯 정답: 지난해 삼 월 김 전 장관의 동료인 장동련 홍익대 교수가 민간 자문단장으로 위촉되면서 본격적인 공모와 개발 작업에 들어갔다
🤖 예측:  지난해 3월 김 전 장관의 동료인 장동량 홍익대 교수가 민간 자문단장으로 위쪽되면서 동격적인 공모와 개발 작업에 들어갔다.

샘플 2:
🎯 정답: 그 바람에 나의 몸도 겹쳐 쓰러지며 한창 피어난 노란 동백꽃 속으로 폭 파묻혀 버렸다
🤖 예측:  그 바람에 나의 몸도 겹쳐스러지면 한창 피어난 노란 동백꽃속으로 폭파무처버렸다.

샘플 3:
🎯 정답: 현재 백화점과 영화관 등은 오픈해 영업하고 있고 테마파크 및 아파트 등의 공사는 이천 십 팔 년 완공을 목표로 진행돼 왔다
🤖 예측:  현재 백화점과 영화관 등은 오픈의 영화파고 있고 테마파크미, 압박트 등의 공사는 2018년 완공을 목표로 진행되어 왔다.

샘플 4:
🎯 정답: 또한 박 전 대통령 측은 앞으로 검찰 수사와 관련해 수사 내용에 대한 입장뿐 아니라 변호인단의 활동 상황 등도 소상하게 공개한다는 방침이다
🤖 예측:  또한 박 전 대통령 측은 앞으로 검찰 수사와 관련해 수사 내용에 대한 입장군 아니라 변호인단의 활동 상황 등도 수상하게 공개한다는 방침입니다.

샘플 5:
🎯 정답: 한 잔의 커피가 소비자에게 전해지기 위해선 여러 차례의 공정을 거쳐야 한다
🤖 예측:  한잔의 커피가 속이 짧게 전해지기 위해서는 여러 차례의 공정을 거쳐야 한다.

샘플 6:
🎯 정답: 다른 지역에서는 그곳이 아무리 넓다 하더라도 푸야 라이몬디를 발견하기가 어려울 것입니다
🤖 예측:  다른 지역에서는 그곳이 아무리 넓다 하더라도 푸야 라인 문제를 발견하기가 어려울 것입니다.

샘플 7:
🎯 정답: 물론 십 대 자녀가 잘못된 결론을 내리거나 심지어 당신과 비슷한 실수를 저질렀을 때 자신을 정당화하지는 않을까 염려가 될 수도 있습니다
🤖 예측:  물론 10대 자녀가 잘못된 결론을 내리거나 심